In [3]:
import pandas as pd
import os
import re

In [4]:
directory = os.getcwd()
gender_submission = "gender_submission.csv"
test = "test.csv"
train = "train.csv"
gender_submission_path = os.path.join(directory, gender_submission)
test_path = os.path.join(directory, test)
train_path = os.path.join(directory, train)
gender_submission_df = pd.read_csv(gender_submission_path)
test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)

In [5]:
df_people = pd.concat([train_df, test_df], join="outer")
df_people = df_people.set_index("PassengerId")

In [6]:
gender_submission_df = gender_submission_df.set_index("PassengerId")
df_people.update(gender_submission_df)

In [7]:
df_people.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1. ¿Cuántas personas iban en el titanic?

In [8]:
df_people["Name"].count()

1309

### Hay 1309 nombres distintos. Eso significa que había 1309 personas en el titanic

## 2. ¿Cuántos hombres y mujeres sobrevivieron?

In [9]:
df_people.groupby(["Survived", "Sex"])["Name"].count()

Survived  Sex   
0.0       female     81
          male      734
1.0       female    385
          male      109
Name: Name, dtype: int64

### Suponiendo que 0 significa que no sobrevivieron (False) y 1 que sí (True), hubo 385 mujeres y 109 hombres que sobrevivieron

## 3. ¿Cuál fue el top 10 de edad que más sobrevieron y el top 10 de edad que no lo lograron?

Dos métodos para obtener esta información. La primera es utilizando las propiedades de pandas y la segunda es mediante agrupaciones.

In [10]:
df_people.loc[df_people['Survived'] == 1]['Age'].value_counts().head(10)

22.0    21
24.0    20
18.0    16
36.0    16
30.0    16
27.0    15
35.0    14
29.0    13
45.0    12
19.0    12
Name: Age, dtype: int64

In [11]:
df_people_groupby_survived_ages = df_people.groupby(["Survived", "Age"])["Name"].count().reset_index(name="count")

In [12]:
df_not_survivers = df_people_groupby_survived_ages.loc[df_people_groupby_survived_ages["Survived"]==0.0]
df_not_survivers.sort_values(by="count", ascending=False).head(10)

,Survived,Age,count
26,0.0,21.0,33
33,0.0,25.0,27
31,0.0,24.0,27
40,0.0,30.0,24
37,0.0,28.0,23
21,0.0,18.0,23
27,0.0,22.0,22
34,0.0,26.0,20
23,0.0,19.0,17
39,0.0,29.0,17


### El top 10 de sobrevivientes es: 22, 24, 18, 36, 30,27, 35, 29, 45 y 19

### El top 10 de no sobrevivientes es: 21, 25, 24, 30, 28, 18, 22, 26, 19 y 29

### Cabe destacar 12 personas cuya edad es menor de 1 con decimales. Se asume que esto es debido a que son personas de apenas unos meses. Por si acaso, se analizan a continuación:

In [13]:
babies = df_people.loc[df_people['Age'] < 1]

In [14]:
babies['Survived'].value_counts()

1.0    9
0.0    3
Name: Survived, dtype: int64

# 4. ¿Cuántos cargos o títulos iban en el barco? Ejemplo: Capitanes, Mrs. Miss, etc. (pista: usa expresiones regulares)

Este ejercicio se podría hacer también de dos modos: por expresiones regulares (como indica el enunciado) y mediante splits a partir de la coma y hasta el punto. Sí es cierto que la segunda opción falla en el único caso de "the Countess", ya que también toma el artículo como título. Por esa razón, se opta únicamente por una expresión regular.

In [15]:
re.findall(r'[A-Z]\w*?\.', 'Failed at Rothes, the Countess. of (Lucy Noel Martha Dyer-Edwards)')

['Countess.']

In [16]:
def extract_title(name):
    try:
        title = re.findall(r'[A-Z]\w*?\.', name)[0]
    except Exception:
        print(f'Failed at {name}')
        title = None
    return title

In [17]:
df_people["Title"] = df_people["Name"].apply(extract_title)

In [18]:
df_people["Title"].value_counts()

Mr.          757
Miss.        260
Mrs.         197
Master.       61
Rev.           8
Dr.            8
Col.           4
Ms.            2
Major.         2
Mlle.          2
Mme.           1
Dona.          1
Lady.          1
Countess.      1
Sir.           1
Capt.          1
Jonkheer.      1
Don.           1
Name: Title, dtype: int64

### En el barco iban: 747 Mr., 260 Miss., 197 Mrs., 61 Master., 8 Rev., 8 Dr., 4 Col., 2 Ms., 2 Major., 2 Mlle., 1 Sir., 1 Jonhkeer., 1 Lady., 1 Dona., 1 Don., 1 Countess., 1 Mme. y 1 Capt.

 # 5. ¿Cuánto es la sumatorio del valor de los tickets en EUR? (pista: los tickets se vendieron en libras esterlinas)

In [20]:
df_people["Fare"].dtype

dtype('float64')

In [29]:
sum_tickets_GBP = df_people["Fare"].sum()
print(sum_tickets_GBP)

43550.4869


Para hacer el cambio de divisas, se ha utilizado el cambio de 1 libra esterlina 1,1703739 euros de acuerdo con https://www.xe.com/es/currencyconverter/convert/?Amount=1&From=GBP&To=EUR a 27/09/2021

In [26]:
sum_tickets_EUR = sum_tickets_GBP * 1.1703739

In [27]:
print(sum_tickets_EUR)

50970.35320005191


### El sumatorio del valor de los tickets en EUR es de 50970,35.